In [10]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

start_time = time.time()

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)
    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()
B_DF = DTBS_DataFrame['B'].copy()
D_DF = DTBS_DataFrame['D'].copy()

In [11]:
# 设置周一（或其他时间）换仓
start_day = '2023-06-01'
end_day = '2023-08-11'

set_which_day = 'wed'
name = '高ytm（全量数据DTBS，用于回测）.xlsx'

"""
按需选择需要分析的策略：
('低价小市值5（全量数据DTBS，用于回测）.xlsx', 'mon')
('高ytm（全量数据DTBS，用于回测）.xlsx', 'wed')
('区间双低3（全量数据DTBS，用于回测）.xlsx', 'thu')
('双低5（全量数据DTBS，用于回测）.xlsx', 'thu')
('小市值3双低5（全量数据DTBS，用于回测）.xlsx', 'wed')
('热门可转债+双低（全量数据DTBS，用于回测）.xlsx', 'mon')
('低价5（全量数据DTBS，用于回测）.xlsx', 'mon')
"""


df = pd.read_excel('pre_data_for_backtest\\'+ name)
print(df)
trade_date_list = sorted(list(D_DF.loc[(D_DF['date'] >= start_day)&(D_DF['date'] <= end_day)&(D_DF['which_day'] == set_which_day),'date']))
print(trade_date_list)

output_df = df[df['日期'].isin(trade_date_list[0:-1])][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
output_df.rename(columns={'日期' : '买入日期'}, inplace=True)
output_df['卖出日期'] = output_df['买入日期'].apply(lambda x:trade_date_list[trade_date_list.index(x)+1])
output_df['持仓周期'] = output_df['买入日期'] + '至' + output_df['卖出日期']
temp = pd.merge(output_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
output_df['可转债名称'] = list(temp['cn'])
output_df['正股代码'] = list(temp['sc'])
output_df['正股名称'] = list(temp['sn'])
output_df['买入价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
output_df['卖出价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '卖出日期'],right_on=['bond_code', 'date'],how='left')['dp'])
output_df['收益率'] = output_df['卖出价'] / output_df['买入价'] - 1
output_df['买卖价差'] = output_df['卖出价'] - output_df['买入价']
# 使用transform方法计算每个分组内的总和
grouped_sum = output_df.groupby('买入日期')['买卖价差'].transform('sum')

# 计算每个分组的百分比
output_df['对于整体收益率的贡献'] = output_df['买卖价差'] / grouped_sum

output_df = output_df[['持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价', '收益率', 
                       '对于整体收益率的贡献', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
print(output_df)

strategy_name = str(name[ : name.index('（')])
mapping = {'mon' : '周一', 
           'tue' : '周二', 
           'wed' : '周三', 
           'thu' : '周四', 
           'fri' : '周五'}
output_excel_name = strategy_name + '（换仓日：' + mapping[set_which_day] + '，用于分析具体转债对整体收益率的影响）.xlsx'
output_df.to_excel('data_for_analyze\\' + output_excel_name, index=False)

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")

          可转债代码          日期    转股溢价率  价格（收盘价全价）   债券余额（亿）       换手率   到期收益率  \
0     123032.SZ  2020-01-02  22.4183    102.428  1.802900  4.233402  4.7704   
1     128042.SZ  2020-01-02  43.1789    104.000  4.158700  3.297900  2.6397   
2     123013.SZ  2020-01-02  29.0239    103.695  0.383000  3.686600  2.5491   
3     113546.SH  2020-01-02  14.1187    105.150  2.299300  3.378000  2.5146   
4     123024.SZ  2020-01-02  25.2961    102.300  2.098905  3.507591  2.5077   
...         ...         ...      ...        ...       ...       ...     ...   
4354  113606.SH  2023-08-10  61.2300    115.347  5.999000  3.590000  1.2600   
4355  123082.SZ  2023-08-11  45.9700    115.700  4.993000  3.210000  1.2600   
4356  128116.SZ  2023-08-11  86.2100    109.990  6.491000  5.570000  1.0800   
4357  123151.SZ  2023-08-11  38.5300    115.460  6.998000  2.400000  1.0200   
4358  127055.SZ  2023-08-11  44.3300    115.620  5.766000  4.080000  0.8400   

       剩余期限（年）        双低  ia  ...  xx15  xx30   hs 

In [6]:
# 查看所有换仓日的情况
start_day = '2023-06-01'
end_day = '2023-08-11'

set_which_days = ['mon', 'tue', 'wed', 'thu', 'fri']

name = '高ytm（全量数据DTBS，用于回测）.xlsx'

"""
按需选择需要分析的策略：
('低价小市值5（全量数据DTBS，用于回测）.xlsx', 'mon')
('高ytm（全量数据DTBS，用于回测）.xlsx', 'wed')
('区间双低3（全量数据DTBS，用于回测）.xlsx', 'thu')
('双低5（全量数据DTBS，用于回测）.xlsx', 'thu')
('小市值3双低5（全量数据DTBS，用于回测）.xlsx', 'wed')
('热门可转债+双低（全量数据DTBS，用于回测）.xlsx', 'mon')
('低价5（全量数据DTBS，用于回测）.xlsx', 'mon')
"""
mapping = {'mon' : '周一', 
           'tue' : '周二', 
           'wed' : '周三', 
           'thu' : '周四', 
           'fri' : '周五'}

df = pd.read_excel('pre_data_for_backtest\\'+ name)
print(df)

all_output_df = []

for set_which_day in set_which_days:
    
    trade_date_list = sorted(list(D_DF.loc[(D_DF['date'] >= start_day)&(D_DF['date'] <= end_day)&(D_DF['which_day'] == set_which_day),'date']))
    print(trade_date_list)

    output_df = df[df['日期'].isin(trade_date_list[0:-1])][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    output_df.rename(columns={'日期' : '买入日期'}, inplace=True)
    output_df['卖出日期'] = output_df['买入日期'].apply(lambda x:trade_date_list[trade_date_list.index(x)+1])
    output_df['持仓周期'] = output_df['买入日期'] + '至' + output_df['卖出日期']
    temp = pd.merge(output_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
    output_df['可转债名称'] = list(temp['cn'])
    output_df['正股代码'] = list(temp['sc'])
    output_df['正股名称'] = list(temp['sn'])
    output_df['买入价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    output_df['卖出价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '卖出日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    output_df['收益率'] = output_df['卖出价'] / output_df['买入价'] - 1
    output_df['买卖价差'] = output_df['卖出价'] - output_df['买入价']
    # 使用transform方法计算每个分组内的总和
    grouped_sum = output_df.groupby('买入日期')['买卖价差'].transform('sum')

    # 计算每个分组的百分比
    output_df['对于整体收益率的贡献'] = output_df['买卖价差'] / grouped_sum
    
    output_df['换仓日'] = mapping[set_which_day]
    
    output_df = output_df[['换仓日', '持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价', '收益率', 
                           '对于整体收益率的贡献', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    
    
    
    ouput_last_df = df[df['日期']==trade_date_list[-1]][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    ouput_last_df.rename(columns={'日期' : '买入日期'}, inplace=True)
    temp = pd.merge(ouput_last_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
    ouput_last_df['可转债名称'] = list(temp['cn'])
    ouput_last_df['正股代码'] = list(temp['sc'])
    ouput_last_df['正股名称'] = list(temp['sn'])
    ouput_last_df['买入价'] = list(pd.merge(ouput_last_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    ouput_last_df['换仓日'] = mapping[set_which_day]
    
    output_df = pd.concat([output_df, ouput_last_df], ignore_index=True)
    
    all_output_df.append(output_df)
    print(output_df)

output_df = pd.concat(all_output_df, ignore_index=True)



output_df.sort_values(by=['买入日期','可转债代码'],ascending=[True,True] ,inplace=True)

strategy_name = str(name[ : name.index('（')])

output_excel_name = strategy_name + '（换仓日：周一至周五，用于分析具体转债对整体收益率的影响）.xlsx'
output_df.to_excel('data_for_analyze\\' + output_excel_name, index=False)

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")

          可转债代码          日期    转股溢价率  价格（收盘价全价）   债券余额（亿）       换手率   到期收益率  \
0     123032.SZ  2020-01-02  22.4183    102.428  1.802900  4.233402  4.7704   
1     128042.SZ  2020-01-02  43.1789    104.000  4.158700  3.297900  2.6397   
2     123013.SZ  2020-01-02  29.0239    103.695  0.383000  3.686600  2.5491   
3     113546.SH  2020-01-02  14.1187    105.150  2.299300  3.378000  2.5146   
4     123024.SZ  2020-01-02  25.2961    102.300  2.098905  3.507591  2.5077   
...         ...         ...      ...        ...       ...       ...     ...   
4368  113595.SH  2023-08-11   3.7700    109.096  2.707000  9.220000  4.2300   
4369  123082.SZ  2023-08-11  45.9700    115.700  4.993000  3.210000  1.2600   
4370  128116.SZ  2023-08-11  86.2100    109.990  6.491000  5.570000  1.0800   
4371  123151.SZ  2023-08-11  38.5300    115.460  6.998000  2.400000  1.0200   
4372  127055.SZ  2023-08-11  44.3300    115.620  5.766000  4.080000  0.8400   

       剩余期限（年）        双低  ia  ...  xx15  xx30   hs 

   换仓日                   持仓周期        买入日期        卖出日期      可转债代码 可转债名称  \
0   周二  2023-06-06至2023-06-13  2023-06-06  2023-06-13  123082.SZ  北陆转债   
1   周二  2023-06-06至2023-06-13  2023-06-06  2023-06-13  128125.SZ  华阳转债   
2   周二  2023-06-06至2023-06-13  2023-06-06  2023-06-13  123153.SZ  英力转债   
3   周二  2023-06-06至2023-06-13  2023-06-06  2023-06-13  113625.SH  江山转债   
4   周二  2023-06-06至2023-06-13  2023-06-06  2023-06-13  113665.SH  汇通转债   
5   周二  2023-06-13至2023-06-20  2023-06-13  2023-06-20  113624.SH  正川转债   
6   周二  2023-06-13至2023-06-20  2023-06-13  2023-06-20  123153.SZ  英力转债   
7   周二  2023-06-13至2023-06-20  2023-06-13  2023-06-20  123144.SZ  裕兴转债   
8   周二  2023-06-13至2023-06-20  2023-06-13  2023-06-20  113569.SH  科达转债   
9   周二  2023-06-13至2023-06-20  2023-06-13  2023-06-20  118008.SH  海优转债   
10  周二  2023-06-20至2023-06-27  2023-06-20  2023-06-27  128125.SZ  华阳转债   
11  周二  2023-06-20至2023-06-27  2023-06-20  2023-06-27  123153.SZ  英力转债   
12  周二  2023-06-20至2023-06-27  2023-06

   换仓日                   持仓周期        买入日期        卖出日期      可转债代码  可转债名称  \
0   周三  2023-06-07至2023-06-14  2023-06-07  2023-06-14  128118.SZ   瀛通转债   
1   周三  2023-06-07至2023-06-14  2023-06-07  2023-06-14  113625.SH   江山转债   
2   周三  2023-06-07至2023-06-14  2023-06-07  2023-06-14  123151.SZ   康医转债   
3   周三  2023-06-07至2023-06-14  2023-06-07  2023-06-14  118016.SH   京源转债   
4   周三  2023-06-07至2023-06-14  2023-06-07  2023-06-14  123109.SZ   昌红转债   
5   周三  2023-06-14至2023-06-21  2023-06-14  2023-06-21  128125.SZ   华阳转债   
6   周三  2023-06-14至2023-06-21  2023-06-14  2023-06-21  113629.SH   泉峰转债   
7   周三  2023-06-14至2023-06-21  2023-06-14  2023-06-21  113668.SH   鹿山转债   
8   周三  2023-06-14至2023-06-21  2023-06-14  2023-06-21  127055.SZ   精装转债   
9   周三  2023-06-14至2023-06-21  2023-06-14  2023-06-21  113657.SH  再22转债   
10  周三  2023-06-21至2023-06-28  2023-06-21  2023-06-28  128125.SZ   华阳转债   
11  周三  2023-06-21至2023-06-28  2023-06-21  2023-06-28  113573.SH   纵横转债   
12  周三  2023-06-21至2023-0

   换仓日                   持仓周期        买入日期        卖出日期      可转债代码 可转债名称  \
0   周四  2023-06-01至2023-06-08  2023-06-01  2023-06-08  128138.SZ  侨银转债   
1   周四  2023-06-01至2023-06-08  2023-06-01  2023-06-08  118020.SH  芳源转债   
2   周四  2023-06-01至2023-06-08  2023-06-01  2023-06-08  118010.SH  洁特转债   
3   周四  2023-06-01至2023-06-08  2023-06-01  2023-06-08  113569.SH  科达转债   
4   周四  2023-06-01至2023-06-08  2023-06-01  2023-06-08  123153.SZ  英力转债   
5   周四  2023-06-08至2023-06-15  2023-06-08  2023-06-15  128116.SZ  瑞达转债   
6   周四  2023-06-08至2023-06-15  2023-06-08  2023-06-15  113584.SH  家悦转债   
7   周四  2023-06-08至2023-06-15  2023-06-08  2023-06-15  127041.SZ  弘亚转债   
8   周四  2023-06-08至2023-06-15  2023-06-08  2023-06-15  118010.SH  洁特转债   
9   周四  2023-06-08至2023-06-15  2023-06-08  2023-06-15  113625.SH  江山转债   
10  周四  2023-06-15至2023-06-29  2023-06-15  2023-06-29  128116.SZ  瑞达转债   
11  周四  2023-06-15至2023-06-29  2023-06-15  2023-06-29  128125.SZ  华阳转债   
12  周四  2023-06-15至2023-06-29  2023-06

   换仓日                   持仓周期        买入日期        卖出日期      可转债代码 可转债名称  \
0   周五  2023-06-02至2023-06-09  2023-06-02  2023-06-09  128063.SZ  未来转债   
1   周五  2023-06-02至2023-06-09  2023-06-02  2023-06-09  123093.SZ  金陵转债   
2   周五  2023-06-02至2023-06-09  2023-06-02  2023-06-09  123153.SZ  英力转债   
3   周五  2023-06-02至2023-06-09  2023-06-02  2023-06-09  113569.SH  科达转债   
4   周五  2023-06-02至2023-06-09  2023-06-02  2023-06-09  123082.SZ  北陆转债   
5   周五  2023-06-09至2023-06-16  2023-06-09  2023-06-16  118020.SH  芳源转债   
6   周五  2023-06-09至2023-06-16  2023-06-09  2023-06-16  123082.SZ  北陆转债   
7   周五  2023-06-09至2023-06-16  2023-06-09  2023-06-16  123153.SZ  英力转债   
8   周五  2023-06-09至2023-06-16  2023-06-09  2023-06-16  113569.SH  科达转债   
9   周五  2023-06-09至2023-06-16  2023-06-09  2023-06-16  113629.SH  泉峰转债   
10  周五  2023-06-16至2023-06-30  2023-06-16  2023-06-30  123153.SZ  英力转债   
11  周五  2023-06-16至2023-06-30  2023-06-16  2023-06-30  113573.SH  纵横转债   
12  周五  2023-06-16至2023-06-30  2023-06